In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys
f = open('gdrive/My Drive/Grokking/reviews.txt')
raw_reviews = f.readlines()
f.close()
f = open('gdrive/My Drive/Grokking/labels.txt')
raw_labels = f.readlines()
f.close()

tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i
    
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))
    
target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [0]:
print(len(raw_reviews))

25000


In [0]:
print(len(raw_labels))

25000


In [0]:
print("tokens len", len(tokens),tokens[0])

tokens len 25000 {'', 'expect', 'comedy', 'isn', 'i', 'inspector', 'financially', 'think', 'ran', 'in', 'teaching', 'and', 'believe', 'to', 'at', 'remind', 'when', 'life', 't', 'other', 'what', 'that', 'high', 'fetched', 'schools', 'burn', 'situation', 'closer', 'tried', 'profession', 'welcome', 'scramble', 'reality', 'than', 'programs', 'immediately', 'same', 'is', 'far', 'me', 'age', 'students', 'repeatedly', 'my', 'cartoon', 'many', 'such', 'sack', 'as', 'classic', 'lead', 'here', 'their', 'pomp', 'your', 'right', 'pathetic', 'years', 'some', '\n', 'pity', 'satire', 'bromwell', 'm', 'much', 's', 'pettiness', 'down', 'about', 'survive', 'time', 'see', 'adults', 'who', 'recalled', 'insightful', 'line', 'school', 'which', 'knew', 'all', 'saw', 'the', 'whole', 'of', 'one', '.', 'through', 'student', 'a', 'teachers', 'episode', 'can', 'it'}


In [0]:
print("len(vocab)", len(vocab),"\n",vocab[:10])

len(vocab) 74074 
 ['miser', 'mellifluous', 'challengers', 'opportunistic', 'melted', 'toenails', 'seaquest', 'supermarkets', 'phillips', 'uninspriring']


In [0]:
#print(word2index)

In [0]:
print(len(input_dataset),input_dataset[0:2])

25000 [[70658, 8713, 16406, 36388, 41514, 43056, 13369, 13882, 39488, 63555, 38467, 1091, 19014, 69193, 66124, 41554, 4692, 6256, 32370, 41082, 60026, 21127, 11402, 38541, 59551, 2724, 43176, 50856, 69290, 71851, 2224, 177, 65202, 9907, 6839, 70329, 66234, 50368, 64196, 53451, 72399, 29405, 15588, 53993, 44787, 63225, 48382, 68864, 41736, 27403, 33036, 20238, 49429, 30490, 42795, 8504, 64831, 72523, 30541, 40787, 40276, 3418, 16739, 29540, 28517, 38774, 15229, 61314, 52612, 71045, 31121, 5528, 70558, 4016, 63408, 947, 67520, 73666, 62406, 58825, 41423, 33234, 11221, 62422, 31707, 30181, 42470, 18922, 7664, 42995, 73205, 61944, 56318], [8713, 50186, 39440, 45588, 5655, 69145, 34330, 11805, 42528, 59937, 36901, 39973, 21029, 11817, 13369, 53820, 70722, 55895, 46695, 32370, 73855, 19593, 40074, 11402, 56971, 38541, 46739, 36503, 50856, 61611, 6832, 2739, 63156, 70329, 50368, 63171, 42184, 48334, 72399, 33999, 26318, 18642, 56025, 20698, 1755, 72925, 63225, 37117, 68864, 70915, 35591, 4173

In [0]:
print(len(target_dataset),target_dataset[0:2])

25000 [1, 0]


In [20]:
import numpy as np
np.random.seed(1)
def sigmoid(x):
    return 1/(1 + np.exp(-x))
alpha, iterations = (0.01, 2)
hidden_size = 100
weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1

correct,total = (0,0)
for iter in range(iterations):
    
    for i in range(len(input_dataset)-1000):
        x,y = (input_dataset[i],target_dataset[i])
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
        
        layer_2_delta = layer_2 - y
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)
        
        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha

        if(np.abs(layer_2_delta) < 0.5):
            correct += 1
        total += 1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter:'+str(iter)\
                              +' Progress:'+progress[2:4]\
                              +'.'+progress[4:6]\
                              +'% Training Accuracy:'\
                              + str(correct/float(total)) + '%')
    print()
    
    

correct,total = (0,0)
for i in range(len(input_dataset)-1000,len(input_dataset)):
  
    x = input_dataset[i]
    y = target_dataset[i]

    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))

    if(np.abs(layer_2 - y) < 0.5):
        correct += 1
    total += 1
print("Test Accuracy:" + str(correct / float(total)))

       

Iter:0 Progress:95.99% Training Accuracy:0.8312916666666667%
Iter:1 Progress:95.99% Training Accuracy:0.865625%
Test Accuracy:0.846


In [31]:

from collections import Counter
import math 

def similar(target='beautiful'):
    target_index = word2index[target]
    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - (weights_0_1[target_index])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))

    return scores.most_common(100)

print("beautiful",word2index['beautiful'])

beautiful 7306


In [32]:
print(similar('beautiful'))

[('beautiful', -0.0), ('steals', -0.712444895381058), ('outstanding', -0.7228375182789654), ('touching', -0.7228499134982306), ('impact', -0.7262514738480319), ('heart', -0.729110280682466), ('liked', -0.7368892254256248), ('friendship', -0.7437019792713794), ('will', -0.746504773680074), ('fun', -0.761538132428488), ('hilarious', -0.762384794339062), ('surprisingly', -0.7629685158180615), ('finest', -0.7631667045548164), ('joan', -0.7643111387510793), ('effective', -0.7650664166273712), ('genre', -0.7685490288344919), ('entertaining', -0.769033454882479), ('subtle', -0.7707830135511047), ('tight', -0.7722575531744053), ('remember', -0.774484272891571), ('different', -0.7766618607593905), ('thank', -0.777231051583139), ('easy', -0.7773736252773664), ('superior', -0.778629744987593), ('favorites', -0.7787002885343809), ('surprised', -0.7790415108474518), ('hooked', -0.7804828088700786), ('simple', -0.7813481347116527), ('true', -0.7814343144962572), ('charlie', -0.7817568659736203), ('b

In [33]:
print(similar('terrible'))

[('terrible', -0.0), ('fails', -0.7241382840924647), ('worse', -0.7450760109639525), ('annoying', -0.7497968840051117), ('dull', -0.773634748997314), ('badly', -0.7801180652553023), ('poor', -0.7932070015624804), ('avoid', -0.8045140718423265), ('save', -0.8047983596970169), ('lacks', -0.8058310063766573), ('disappointment', -0.8195627162833875), ('bad', -0.8195731797244422), ('mess', -0.8271075180089617), ('wooden', -0.837303652287524), ('supposed', -0.8518478247786274), ('ridiculous', -0.8597581909122247), ('wasted', -0.8631804466293277), ('boring', -0.869622899568372), ('horrible', -0.8770076920756217), ('laughable', -0.8773361067977283), ('basically', -0.8929949680879719), ('redeeming', -0.8935991283513064), ('disappointed', -0.9146252854740384), ('disappointing', -0.9173211084949977), ('pointless', -0.9176431208592295), ('unfortunately', -0.9178218910196023), ('unless', -0.9232065966975099), ('lame', -0.9280689560544291), ('oh', -0.9317055500201129), ('stupid', -0.943236084091082)